In [1]:
import numpy as np
import pickle as pk
import pandas as pd
import timeit as tm
import csv
import sys

# Opening data

In [2]:
train_dat = pd.read_csv('../data/clean_data/train_vectors.csv', index_col=0, encoding='utf-8')
del train_dat['TYPE']
print train_dat

        YEAR  MONTH   LATITUDE   LONGITUDE  N_VICTORIA-FRASERVIEW  N_WEST_END  \
148134  2006      5  49.283335 -123.112899                      0           0   
148135  2006      5  49.283335 -123.112899                      0           0   
148136  2006      6  49.283335 -123.112899                      0           0   
148137  2006      6  49.283335 -123.112899                      0           0   
148138  2006      6  49.283335 -123.112899                      0           0   
148139  2006      6  49.283335 -123.112899                      0           0   
148140  2006      6  49.283335 -123.112899                      0           0   
148141  2006      6  49.283335 -123.112899                      0           0   
148142  2006      6  49.283335 -123.112899                      0           0   
148143  2006      7  49.283335 -123.112899                      0           0   
148144  2006      8  49.283335 -123.112899                      0           0   
148145  2006      8  49.2833

In [3]:
train_lbl = pd.read_csv('../data/clean_data/train_labels.csv', index_col=0, encoding='utf-8')
del train_lbl['YEAR']
print train_lbl

        BNE Commercial  BNE Residential/Other  Mischief  Other Theft  \
148134               0                      0         0            0   
148135               0                      0         0            0   
148136               0                      0         0            0   
148137               0                      0         0            0   
148138               0                      0         0            0   
148139               0                      0         0            0   
148140               0                      0         0            0   
148141               0                      0         0            0   
148142               0                      0         0            0   
148143               0                      0         0            0   
148144               0                      0         0            0   
148145               0                      0         0            0   
148146               0                      0         0         

In [4]:
headers = [list(train_dat)]
dat_mtx = train_dat.as_matrix()
print dat_mtx.shape
lbl_mtx = train_lbl.as_matrix()
print lbl_mtx.shape

(295169, 64)
(295169, 6)


In [5]:
# Open test data
test_dat = pd.read_csv('../data/clean_data/test_vectors.csv', index_col=0, encoding='utf-8')
del test_dat['TYPE']
print test_dat

        YEAR  MONTH   LATITUDE   LONGITUDE  N_VICTORIA-FRASERVIEW  N_WEST_END  \
444989  2015      2  49.271601 -123.126966                      0           0   
444990  2015      1  49.282477 -123.118136                      0           0   
444991  2015      8  49.282477 -123.118136                      0           0   
444992  2015      9  49.282477 -123.118136                      0           0   
444993  2015      6  49.253906 -123.112624                      0           0   
444994  2015      5  49.223887 -123.129722                      0           0   
444995  2015      3  49.225733 -123.128743                      0           0   
444996  2015     12  49.266030 -123.130657                      0           0   
444997  2015     12  49.208469 -123.132941                      0           0   
444998  2015      3  49.250341 -123.072630                      0           0   
444999  2015      3  49.250107 -123.101081                      0           0   
445000  2015      6  49.2361

In [6]:
test_lbl = pd.read_csv('../data/clean_data/test_labels.csv', index_col=0, encoding='utf-8')
del test_lbl['YEAR']
print test_lbl

        BNE Commercial  BNE Residential/Other  Mischief  Other Theft  \
444989               0                      0         0            0   
444990               0                      0         1            0   
444991               0                      0         1            0   
444992               0                      0         1            0   
444993               0                      0         0            0   
444994               0                      1         0            0   
444995               0                      1         0            0   
444996               0                      0         0            0   
444997               0                      0         0            0   
444998               0                      1         0            0   
444999               0                      0         0            1   
445000               0                      0         0            0   
445001               0                      0         0         

In [7]:
dat_mtx_tst = test_dat.as_matrix()
print dat_mtx_tst.shape
lbl_mtx_tst = test_lbl.as_matrix()
print lbl_mtx_tst.shape

(34142, 64)
(34142, 6)


# Linear Regression

In [8]:
from sklearn.linear_model import LinearRegression

# Fit Linear Regression
lin_reg = LinearRegression(n_jobs=-1)
lin_reg.fit(dat_mtx, lbl_mtx)

/usr/local/lib/python2.7/site-packages/scipy/linalg/basic.py:884: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [9]:
# Generate predictions
predictions = lin_reg.predict(dat_mtx_tst)
print predictions.shape

(34142, 6)


In [10]:
import math

errors = []

# compute squared errors
for i in xrange(predictions.shape[0]):
    p = predictions[i]
    t = lbl_mtx_tst[i]
    
    # compute distance
    squared_distance = 0.0
    for j in xrange(predictions.shape[1]):
        squared_distance += (p[j] - t[j])**2
    
    errors.append(squared_distance)

rmse = math.sqrt(sum(errors)/len(errors))
print 'Root mean squared error:', rmse

Root mean squared error: 0.841330416129


# Logistic regression

In [11]:
txt_train_lbl = []
for lbl in lbl_mtx:
    elem = 0
    
    for i, k in enumerate(lbl):
        elem += i * k
    
    txt_train_lbl.append(elem)

In [12]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(n_jobs=-1)
clf.fit(dat_mtx, txt_train_lbl)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
predictions = clf.predict(dat_mtx_tst)
p_predictions = clf.predict_proba(dat_mtx_tst)

In [14]:
print predictions
print p_predictions

[4 3 3 ..., 4 4 3]
[[  8.22403112e-02   7.33845965e-02   1.21440665e-01   3.39837244e-01
    3.47026052e-01   3.60711310e-02]
 [  3.19871315e-02   7.57400201e-06   5.85319191e-02   8.11980263e-01
    9.52061751e-02   2.28693757e-03]
 [  3.20854566e-02   7.51061450e-06   5.89964527e-02   8.12399488e-01
    9.42557959e-02   2.25529637e-03]
 ..., 
 [  7.32070818e-02   1.18202758e-01   1.30526363e-01   2.80417738e-01
    3.50357859e-01   4.72882006e-02]
 [  4.80865201e-02   2.25853951e-01   1.62800959e-01   1.94216210e-01
    2.75163384e-01   9.38789746e-02]
 [  3.61362065e-02   2.41449711e-05   7.25943517e-02   7.97667382e-01
    8.96999321e-02   3.87798319e-03]]


In [15]:
for i in range(6):
    print str(i)+'\t',

print ''

for i in xrange(len(p_predictions)):
    
    for j in xrange(len(p_predictions[i])):
        print("%.2f" % (p_predictions[i][j]*100))+'%\t',
    
    print ''

0	1	2	3	4	5	
8.22%	7.34%	12.14%	33.98%	34.70%	3.61%	
3.20%	0.00%	5.85%	81.20%	9.52%	0.23%	
3.21%	0.00%	5.90%	81.24%	9.43%	0.23%	
3.21%	0.00%	5.91%	81.24%	9.41%	0.23%	
6.00%	10.70%	12.20%	35.50%	29.19%	6.41%	
5.75%	13.61%	13.79%	27.16%	32.89%	6.81%	
5.85%	14.80%	13.67%	24.60%	33.99%	7.10%	
7.55%	11.00%	12.58%	29.77%	34.32%	4.77%	
5.47%	19.11%	14.52%	18.66%	34.32%	7.91%	
5.63%	16.07%	15.60%	26.13%	28.03%	8.53%	
5.67%	11.91%	13.03%	33.05%	28.78%	7.55%	
5.04%	26.00%	16.88%	15.49%	29.19%	7.39%	
6.56%	1.41%	12.20%	48.92%	28.24%	2.67%	
6.07%	10.93%	12.34%	34.30%	29.78%	6.58%	
7.27%	9.74%	12.33%	33.84%	32.26%	4.57%	
5.69%	13.31%	15.25%	30.84%	26.89%	8.03%	
7.45%	10.19%	12.55%	31.10%	33.90%	4.82%	
6.49%	11.75%	13.35%	30.20%	32.63%	5.58%	
6.42%	9.04%	12.80%	34.62%	32.67%	4.46%	
6.07%	10.73%	13.29%	30.40%	33.31%	6.20%	
6.15%	10.95%	13.44%	29.16%	33.96%	6.35%	
7.26%	11.39%	12.46%	31.80%	32.27%	4.82%	
6.11%	12.48%	13.28%	29.46%	32.24%	6.42%	
6.27%	13.04%	16.98%	28.61%	27.30%	7.80%	
6.36%	10.70%	13.

In [16]:
txt_tst_lbl = []
for lbl in lbl_mtx_tst:
    elem = 0
    
    for i, k in enumerate(lbl):
        elem += i * k
    
    txt_tst_lbl.append(elem)

In [17]:
from sklearn.metrics import accuracy_score
score = accuracy_score(txt_tst_lbl, predictions)
print score

0.40642024486
